In [1]:
import numpy as np

In [414]:
class LogisticRegression:
    def __init__(self, lr, sgd=False):
        self.lr = lr
        self.sgd = sgd
        self.loss_vals = []
        self.w = None

    def loss(self, y, y_pred):
        eps = 1e-5 # to avoid log(0) error
        return (-1/y_pred.shape[0]) * np.sum((y * np.log(y_pred+eps) + (1-y) * np.log(1-y_pred+eps)))
    
    def grad(self, y, y_pred, X):
        return (X.T @ (y_pred - y)) / X.shape[0]

    def sigmoid(self, x):
        return 1 / (1+np.exp(-x))

    def fit(self, X, y, num_iters=10):
        # X shape: (n, k)
        # y shape: (n, 1)
        # w shape: (k, 1)
        (n, k) = X.shape
        self.w = np.zeros((k+1, 1))
        
        for _ in range(num_iters):
            if self.sgd:
                self.sgd_fit(X, y)
            else:
                self.batchgd_fit(X, y)

    def add_bias_to_feats(self, X):
        (n, k) = X.shape
        X = np.hstack((X, np.ones((n, 1))))
        return X

    def batchgd_fit(self, X, y):
        X = self.add_bias_to_feats(X)
        y_pred = self.sigmoid(X@self.w)
        y = y.reshape((y.shape[0], 1))
        grad = self.grad(y, y_pred, X)
        self.w -= self.lr * grad
        self.loss_vals.append(self.loss(y_pred, y))

    def sgd_fit(self, X, y):
        loss = 0
        X = self.add_bias_to_feats(X)
        for x, label in zip(X, y):
            y_pred = self.sigmoid(x @ self.w)
            label = np.array([label])
            grad = self.grad(label, y_pred, x.reshape((1, x.shape[0])))
            grad = grad.reshape((x.shape[0], 1))
            self.w -= self.lr * grad
            loss += self.loss(y_pred, label)
        self.loss_vals.append(loss / X.shape[0])
        
    def predict(self, X, y):
        num_samples = X.shape[0]
        X = self.add_bias_to_feats(X)
        y_pred = self.sigmoid(X @ self.w)
        loss = self.loss(y, y_pred)
        return loss / num_samples, np.round(y_pred)


In [415]:
lr = LogisticRegression(0.001, sgd=True)

In [416]:
X = np.array([[4], [4], [4], [0], [0], [0]])
y = np.array([1, 1, 1, 0, 0, 0])
lr.fit(X, y, num_iters=1)
print(lr.predict(X, y))
print(lr.w)

(0.6931628523363192, array([[1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.]]))
[[ 5.97453658e-03]
 [-7.11083104e-06]]


In [442]:
class LinearRegression:
    def __init__(self, lr):
        self.lr = lr
        self.w = None # (k+1, 1), including bias term
        
    def add_bias_to_X(self, X):
        (n, _) = X.shape
        bias_term = np.ones((n, 1))
        return np.hstack((X, bias_term))
    
    def initialize_w(self, k):
        self.w = np.zeros((k, 1))
        
    def loss(self, y, y_pred):
        # MSE
        m = y.shape[0]
#         print(y, y_pred, (1/m) * np.sum(np.square(y_pred - y)))
        return (1/m) * np.sum(np.square(y_pred - y))
    
    def gradient(self, y, y_pred, X_with_bias):
        '''
        Input
        - y: (n, 1)
        - y_pred: (n, 1)
        - X_with_bias: (n, k+1)
        
        Output:
        - grad: (k+1, 1)
        '''
        m = y.shape[0]
        return (2/m) * (X_with_bias.T @ (y_pred - y))
        
    def fit(self, X, y, num_iters=10):
        # X: (n, k)
        # y: (n, 1)
        X_with_bias = self.add_bias_to_X(X)
        self.initialize_w(X_with_bias.shape[1])
        cost_list = []
        for _ in range(num_iters):
            y_pred = X_with_bias @ self.w
            cost = self.loss(y, y_pred)
            gradient = self.gradient(y, y_pred, X_with_bias)
            self.w -= self.lr * gradient
            cost_list.append(cost)
        return cost_list
    
    def predict(self, X, y):
        X_with_bias = self.add_bias_to_X(X)
        y_pred = X_with_bias @ self.w
        cost = self.loss(y, y_pred)
        return cost, np.round(y_pred)
            

In [444]:
lr = LinearRegression(0.001)
X = np.array([[4], [4], [4], [0], [0], [0]])
y = np.array([[5], [5], [5], [0], [0], [0]])
lr.fit(X, y, num_iters=100)
print(lr.predict(X, y))
print(lr.w)

(0.4385097186664136, array([[4.],
       [4.],
       [4.],
       [0.],
       [0.],
       [0.]]))
[[0.96613788]
 [0.22684466]]


In [539]:

class KMeans:
    def __init__(self, k):
        self.k = k
        self.centroids = None
        self.centroids_to_point = [[] for _ in range(self.k)]
        
    def init_centroids(self, X):
        self.centroids = X[np.random.choice(X.shape[0], size=self.k, replace=False)]
        
    def cluster(self, X):
        self.init_centroids(X)
        prev_centroids = None
        i = 0
        while prev_centroids is None or not (self.centroids == prev_centroids).all():
            i += 1
            centroid_sum = np.zeros((self.k, X[0].shape[0]))
            centroid_total = [0 for _ in range(self.k)]
            self.centroids_to_point = [[] for _ in range(self.k)]
            prev_centroids = self.centroids
            for p in X:
                centroid_idx = np.argmin(np.array([np.linalg.norm(p - c) for c in self.centroids]))
                centroid_sum[centroid_idx] += p
                centroid_total[centroid_idx] += 1
                self.centroids_to_point[centroid_idx].append(p.copy())
            # re evaluate centroids
            for i, (c_sum, c_count) in enumerate(zip(centroid_sum, centroid_total)):
                self.centroids[i] = c_sum / c_count
        return self.centroids
        

In [540]:
km = KMeans(5)
X = np.array([[0, 0], [0, 1], [0, 2], [1, 2], [1, 3], [1, 4]])
print(km.cluster(X))


[[1 2]
 [0 0]
 [1 4]
 [1 3]
 [0 2]]


In [583]:
class Perceptron:
    def __init__(self, lr):
        self.lr = lr
        self.w = None
        
    def fit(self, X, y, num_iters=10):
        # X: (n, k)
        # y: (n, 1)
        # w = (k+1, 1)
        
        (n, k) = X.shape
        self.w = np.zeros((k+1, 1))
        X_with_bias = np.hstack((X, np.ones((n, 1))))
        for _ in range(num_iters):
            incorrect = 0
            for x_i, y_i in zip(X_with_bias, y):
                y_pred = (x_i @ self.w)[0]
                if y_i * y_pred <= 0:
                    incorrect += 1
                    self.w += self.lr * y_i * x_i.T.reshape(self.w.shape)
    
    def predict(self, X, y):
        (n, k) = X.shape
        X_with_bias = np.hstack((X, np.ones((n, 1))))
        y_preds = X_with_bias @ self.w
        out = []
        for y_pred in y_preds:
            if y_pred <= 0:
                out.append(-1)
            else:
                out.append(1)
        return out

In [584]:
p = Perceptron(0.1)
X = np.array([[4], [4], [4], [0], [0], [0]])
y = np.array([[1], [1], [1], [-1], [-1], [-1]])
p.fit(X, y, num_iters=10)
p.predict(X, y)

[1, 1, 1, -1, -1, -1]

In [638]:
class KNNClassifier:
    def __init__(self, k):
        self.k = k
        self.train_X = None
        self.train_y = None
        
    def fit(self, X, y):
        self.train_X = X
        self.train_y = y
    
    def euclid_dist(self, p1, p2):
        return np.sum(np.sqrt(np.square(p1 - p2)))
        
    def predict(self, X):
        preds = []
        for r in X:
            out = np.zeros(X.shape[0])
            for i, train_r in enumerate(self.train_X):
                out[i] = self.euclid_dist(train_r, r)
            smallest_k_idx = np.argsort(out)[:self.k]
            smallest_k_labels = self.train_y[smallest_k_idx]
            print(smallest_k_labels)
            print(np.bincount(smallest_k_labels))
            preds.append(np.argmax(np.bincount(smallest_k_labels)))
        return preds

In [642]:
knn_class = KNNClassifier(1)
X = np.array([[4], [4], [4], [0], [0], [0]])
y = np.array([1, 1, 1, 0, 0, 0])
knn_class.fit(X, y)

X = np.array([[4], [3], [2], [1], [0], [-1]])
y = np.array([[1], [1], [1], [-1], [-1], [-1]])
knn_class.predict(X)

[1]
[0 1]
[1]
[0 1]
[1]
[0 1]
[0]
[1]
[0]
[1]
[0]
[1]


[1, 1, 1, 0, 0, 0]

In [661]:
class KNNRegressor:
    def __init__(self, k):
        self.k = k
        self.train_X = None
        self.train_y = None
        
    def fit(self, X, y):
        self.train_X = X
        self.train_y = y
    
    def euclid_dist(self, p1, p2):
        return np.sum(np.sqrt(np.square(p1 - p2)))
        
    def predict(self, X):
        preds = []
        for r in X:
            out = np.zeros(X.shape[0])
            for i, train_r in enumerate(self.train_X):
                out[i] = self.euclid_dist(train_r, r)
            smallest_k_idx = np.argsort(out)[:self.k]
            smallest_k_labels = self.train_y[smallest_k_idx]
            preds.append(np.mean(smallest_k_labels))
        return preds

In [662]:
knn_class = KNNRegressor(1)
X = np.array([[6], [5], [4], [3], [2], [1]])
y = np.array([6, 5, 4, 3, 2, 1])
knn_class.fit(X, y)

X = np.array([[4], [3], [2], [3], [2], [1]])
y = np.array([[1], [1], [1], [-1], [-1], [-1]])
knn_class.predict(X)

[4.0, 3.0, 2.0, 3.0, 2.0, 1.0]